## 1. Data Processing ##

In [30]:
import os
import numpy as np
import pandas as pd

# Chemin des données
data_folder = "cancer/"
cancers = ["aml", "breast", "colon", "gbm", "kidney", "liver", "lung", "melanoma", "ovarian", "sarcoma"]

# region : Définitions des fonctions

# Fonction pour appliquer une log2-transformation
def log2_transform(df, offset=1e-5):
    """Transforme les valeurs en log2(x + offset) pour éviter log(0)."""

    return np.log2(df + offset)

# Fonction pour filtrer les lignes selon l'écart-type et le mode de filtrage
def filter_by_std(df, filtering="none"):
    """Filtre les lignes selon l'écart-type et le mode de filtrage."""

    # Si le filtrage est 'none', on ne filtre pas et retourne les données sans changement
    if filtering == "none":
        print(f"Filtrage 'none' : Toutes les {len(df)} lignes sont conservées.\n")
        return df
    
    # Calcul de l'écart-type pour chaque ligne
    std_devs = df.std(axis=1)
    
    # Tri décroissant des écarts-types
    sorted_std_devs = np.sort(std_devs)[::-1]
    
    # Déterminer le seuil en fonction du mode de filtrage
    if filtering == "stringent":
        threshold_idx = min(len(sorted_std_devs), 5000)
        
    else:  # Mode "standard"
        threshold_idx = min(len(sorted_std_devs), 6000)
    
    # Déterminer la valeur seuil
    threshold_value = sorted_std_devs[threshold_idx - 1]
    print(f"Seuil d'écart-type ({filtering}): {threshold_value}\n")
    
    # Filtrer les lignes avec un écart-type supérieur ou égal au seuil
    filtered_df = df[std_devs >= threshold_value]
    
    # Afficher combien de lignes ont été conservées après filtrage
    print(f"Avant filtrage : {len(df)} lignes. Après filtrage : {len(filtered_df)} lignes.")
    
    return filtered_df

# Fonction pour sélectionner les colonnes communes entre plusieurs matrices
def intersect_columns(*dfs):
    """Trouve les colonnes communes entre plusieurs DataFrames."""

    common_columns = set(dfs[0].columns)

    for df in dfs[1:]:
        common_columns &= set(df.columns)

    return [df[common_columns] for df in dfs]

# Fonction pour charger et préparer les données
def prepare_factorization_data(cancer_path, filtering="none"):

    print(f"## Préparation des données pour : {cancer_path} ##")

    # Chemins des fichiers
    exp_path = os.path.join(cancer_path, "exp")
    mirna_path = os.path.join(cancer_path, "mirna")
    methy_path = os.path.join(cancer_path, "methy")

    # Lire les fichiers
    exp = pd.read_csv(exp_path, sep=" ", index_col=0)
    mirna = pd.read_csv(mirna_path, sep=" ", index_col=0)
    methy = pd.read_csv(methy_path, sep=" ", index_col=0)

    # Log2-transformation pour exp et mirna
    log_exp = log2_transform(exp)
    log_mirna = log2_transform(mirna)

    # Filtrer les lignes selon l'écart-type (si activé)
    log_exp = filter_by_std(log_exp, filtering)
    log_mirna = filter_by_std(log_mirna, filtering)
    methy = filter_by_std(methy, filtering)


    # Rajouter survival 
    survival = pd.read_csv("cancer/aml/survival", sep="\t")
    survival['PatientID'] = survival['PatientID'].str.replace("-", ".")
    survival = survival.set_index("PatientID")
    survival = survival.transpose()
    survival = survival.loc[:, ~survival.columns.duplicated()]
    

    # Sélectionner les colonnes communes entre les matrices
    common_columns = list(set(log_exp.columns) & set(log_mirna.columns) & set(methy.columns))
    print(len(common_columns), "sont les colonnes communes trouvées.\n")

    methy = methy[common_columns]
    log_exp = log_exp[common_columns]
    log_mirna = log_mirna[common_columns]

    # Vérifier l'existence des colonnes dans survival
    available_columns_in_survival = [col for col in common_columns if col in survival.columns]

    if available_columns_in_survival:
        survival = survival[available_columns_in_survival]

    # Combiner les matrices (concaténation)
    combined_data = pd.concat([log_exp, methy, log_mirna, survival], axis=0)
    
    return combined_data

# endregion

# region : Application des fonctions

# Pour chaque type de cancer
for cancer in cancers:

    cancer_path = os.path.join(data_folder, cancer)

    if os.path.exists(cancer_path):

        combined_data = prepare_factorization_data(cancer_path, filtering="standard")  # Ou "stringent", "standard", ou "none"

        # Sauvegarder pour vérification
        save_path = os.path.join(cancer_path, "combined_data.csv")
        combined_data.to_csv(save_path)

        print(f"Les données combinées et traitées ont été sauvegardées dans : {save_path}\n")

    else:
        print(f"Chemin introuvable : {cancer_path}")

# endregion


## Préparation des données pour : cancer/aml ##
Seuil d'écart-type (standard): 3.2830883652259537

Avant filtrage : 20531 lignes. Après filtrage : 6000 lignes.
Seuil d'écart-type (standard): 0.0

Avant filtrage : 705 lignes. Après filtrage : 705 lignes.
Seuil d'écart-type (standard): 0.2834424351022285

Avant filtrage : 5000 lignes. Après filtrage : 5000 lignes.
170 sont les colonnes communes trouvées.

Les données combinées et traitées ont été sauvegardées dans : cancer/aml\combined_data.csv

## Préparation des données pour : cancer/breast ##
Seuil d'écart-type (standard): 2.263280860985602

Avant filtrage : 20531 lignes. Après filtrage : 6000 lignes.
Seuil d'écart-type (standard): 0.0

Avant filtrage : 1046 lignes. Après filtrage : 1046 lignes.
Seuil d'écart-type (standard): 0.23276402649296526

Avant filtrage : 5000 lignes. Après filtrage : 5000 lignes.
685 sont les colonnes communes trouvées.

Les données combinées et traitées ont été sauvegardées dans : cancer/breast\combined_data

## 2. Dataframe Setup

In [33]:
# Pour chaque type de cancer
for cancer in cancers:

    cancer_path = os.path.join('cancer', cancer)
    combined_data_path = os.path.join(cancer_path, 'combined_data.csv')

    # Lire le fichier CSV
    combined_data = pd.read_csv(combined_data_path, sep=",", on_bad_lines="skip")

    # Transposer les données
    final_data = combined_data.transpose()

    # Extraire la première ligne et l'affecter au nom des colonnes
    final_data.columns = final_data.iloc[0]
    final_data = final_data[1:]

    # Effacer les valeurs nulles suite à la transposition
    final_data = final_data.dropna()

    # Réinitialiser les index
    final_data.reset_index(drop=True, inplace=True)

    # Définir le chemin de sauvegarde pour le fichier final
    save_path = os.path.join(cancer_path, "final_data.csv")
    
    # Sauvegarder le fichier CSV final
    final_data.to_csv(save_path, index=False)  # Ne pas inclure l'index dans le CSV

    print(f"Les données de combined_data.csv ont été transposées puis enregistrées dans : {save_path}\n")


Les données de combined_data.csv ont été transposées puis enregistrées dans : cancer\aml\final_data.csv

Les données de combined_data.csv ont été transposées puis enregistrées dans : cancer\breast\final_data.csv

Les données de combined_data.csv ont été transposées puis enregistrées dans : cancer\colon\final_data.csv

Les données de combined_data.csv ont été transposées puis enregistrées dans : cancer\gbm\final_data.csv

Les données de combined_data.csv ont été transposées puis enregistrées dans : cancer\kidney\final_data.csv

Les données de combined_data.csv ont été transposées puis enregistrées dans : cancer\liver\final_data.csv

Les données de combined_data.csv ont été transposées puis enregistrées dans : cancer\lung\final_data.csv

Les données de combined_data.csv ont été transposées puis enregistrées dans : cancer\melanoma\final_data.csv

Les données de combined_data.csv ont été transposées puis enregistrées dans : cancer\ovarian\final_data.csv

Les données de combined_data.csv on

Nous avons transposé les données. Si on souhaite analyser ou visualiser ces données de manière à ce que les échantillons soient en lignes et les gènes en colonnes, il faut effectivement transposer le tableau. Cependant, pour les analyses de type statistique, les lignes doivent souvent représenter les gènes, tandis que les colonnes doivent représenter les échantillons, comme c'est le cas avant notre modification. 

## 3. Data Checkup ##

In [34]:
# Pour chaque type de cancer
for cancer in cancers:

    cancer_path = os.path.join('cancer', cancer)
    final_data_path = os.path.join(cancer_path, 'final_data.csv')

    data = pd.read_csv(final_data_path, sep=",", on_bad_lines="skip")

    print(f"Check info on : {final_data_path}\n")

    # Vérifier les types de données
    print(f"Type de données :")
    print(data.dtypes) 

    # Vérifier les statistiques descriptives
    print(f"\nStatistiques :")
    print(data.describe())  

    # Vérifier les valeurs manquantes
    print(f"\nValeurs nulles manquantes :")
    print(data.isnull().sum())  

    print("\n" + "-"*50 + "\n")


Check info on : cancer\aml\final_data.csv

Type de données :
X..100130426    float64
X..100133144    float64
X..100134869    float64
X..26823        float64
X..340602       float64
                 ...   
hsa.mir.98      float64
hsa.mir.99a     float64
hsa.mir.99b     float64
Survival        float64
Death           float64
Length: 11707, dtype: object

Statistiques :
       X..100130426  X..100133144  X..100134869    X..26823   X..340602  \
count    159.000000    159.000000    159.000000  159.000000  159.000000   
mean     -13.468205      0.755949      2.064049   -0.454793   -6.814063   
std        6.540969      6.988206      4.807799    5.707779    8.763527   
min      -16.609640    -16.609640    -16.609640  -16.609640  -16.609640   
25%      -16.609640      1.287526      1.978526    0.367805  -16.609640   
50%      -16.609640      3.170215      3.269932    1.377074   -0.499993   
75%      -16.609640      4.678137      4.041346    2.248148    0.828307   
max        1.923952      7.237

# END